In [ ]:
# モデルの URI とコンテナイメージの取得
import boto3
sm_client = boto3.client('sagemaker')
endpoint_config_name = sm_client.describe_endpoint(EndpointName='jumpstart-ftc-kinoko-detection-endpoint')['EndpointConfigName']
model_name = sm_client.describe_endpoint_config(EndpointConfigName=endpoint_config_name)['ProductionVariants'][0]['ModelName']
model_detail = sm_client.describe_model(ModelName=model_name)
model_container_image = model_detail['PrimaryContainer']['Image']
model_s3_uri = model_detail['PrimaryContainer']['ModelDataUrl']
print(model_container_image)
print(model_s3_uri)

In [ ]:
# モデルのダウンロード
!rm -rf model/
!mkdir -p model/
!aws s3 cp {model_s3_uri} ./model/
%cd model/
!tar zxvf model.tar.gz

In [ ]:
# 推論準備
%cd code
!pip install sagemaker-inference

In [ ]:
# 依存しているライブラリをインストール
!pip install sagemaker-inference

import json, inference
from PIL import Image, ImageDraw, ImageFont

# モデルのロード
model = inference.model_fn('../')
# 推論に使う画像ファイル
IMAGE_FILE = '../../../test_raw_images/lattice.jpg'
# 画像ファイルをバイナリとして開く
with open(IMAGE_FILE,'rb') as f:
    img_bin = f.read()
# 推論
pred = json.loads(inference.transform_fn(model,img_bin,'application/x-image','application/json'))
# 推論結果の展開
bboxes, classes, confidences = pred['normalized_boxes'], pred['classes'], pred['scores']
# 推論結果の描画
img = Image.open(IMAGE_FILE)
draw = ImageDraw.Draw(img)
for i in range(len(bboxes)):
    # 信頼度スコアが0.8以上のみ描画する
    if confidences[i]>0.8:
        # 矩形の相対座標を取得
        left, top, right, bottom = bboxes[i]
        # 矩形の相対座標を絶対座標に変換
        left = img.size[0] * left
        top = img.size[1] * top
        right = img.size[0] * right
        bottom = img.size[1] * bottom
        # 検出した物体の ID を take/kino に読み替える
        text = 'take' if int(classes[i])==0 else 'kino'
        # take/kinoに対して矩形で描画するための色を設定する
        color = 'blue' if text == 'take' else 'red'
        # 矩形の左上に表示する文字の大きさを設定、きのこの山なら大きくする
        TEXTSIZE=14 if classes[i]=='0' else 18
        # 矩形の先の太さを設定、きのこの山なら太くする
        LINEWIDTH=4 if classes[i]=='0' else 6
        # 矩形を描画する
        draw.rectangle([(left,top),(right,bottom)], outline=color, width=LINEWIDTH)
        # 矩形の左上に描画する信頼度スコアの取得
        text += f' {str(round(confidences[i],3))}'
        # テキストを描画する場所を取得
        txpos = (left, top-TEXTSIZE-LINEWIDTH//2)
        # フォントの設定
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSerif.ttf", size=TEXTSIZE)
        # 描画するテキストのサイズを取得
        txw, txh = draw.textsize(text, font=font)
        # テキストの背景用の矩形を描画
        draw.rectangle([txpos, (left+txw, top)], outline=color, fill=color, width=LINEWIDTH)
        # テキストを描画
        draw.text(txpos, text, fill='white',font=font)
# 画像をファイルに書き出す
img.save('pred_lattice.png')

In [ ]:
img